In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

!pip install transformers
!pip install torch

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

import random
import numpy as np
import pandas as pd
# pytorch 사용할거라 tensorflow 임포트 안함

rand_seed = 42
random.seed(rand_seed)
np.random.seed(rand_seed)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

print(torch.__version__)

1.12.1+cu113


In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
path = "gdrive/My Drive/Colab Notebooks/mbti"

In [ ]:
os.listdir("gdrive/My Drive/Colab Notebooks/mbti")

['test.csv',
 'train.csv',
 '3epoch_tpu.h5',
 'mbti_model.h5',
 'checkBert.h5',
 'check111111111.h5',
 'checkBert12.h5',
 'mbti_model12.h5',
 'mbti_model5350.h5',
 'inferencetorch.pt',
 'fulltorch.pt',
 'lastEpochModel64.h5',
 'bestBert64.h5',
 'mbti_oh.csv',
 'mbti_oh_num.csv',
 'kcembtiweight1.pt',
 'kcembtifullmodel1.pt',
 'koembtiweight1.pt',
 'koembtifullmodel1.pt']

In [ ]:
mbti = pd.read_csv(os.path.join(path,"mbti_oh_num.csv"))
train = pd.read_csv(os.path.join(path,"train.csv"))
test = pd.read_csv(os.path.join(path,"test.csv"))

In [ ]:
# 모델의 설정이 정수형 라벨을 필요로 해서 int형 label 컬럼을 생성 (모델 로딩시 num_labels=16 설정)
# - 내부적으로 label에 대해 원핫 인코딩 작업 수행함
train['label'] = train['category'].apply(lambda v: v)
test['label'] = test['category'].apply(lambda v: v)

# [0~4] label 확인 : num_classes=16
unq_labels, cnt_labels = np.unique(train['label'].values, return_counts=True)
print('labels =', dict(zip( unq_labels, cnt_labels )) )

train[['category','label']].dtypes

dataset_train = train
dataset_valid = test

labels = {0: 2386, 1: 3397, 2: 2469, 3: 4023, 4: 1578, 5: 1829, 6: 1666, 7: 3233, 8: 2470, 9: 2511, 10: 3390, 11: 3193, 12: 3524, 13: 2989, 14: 3506, 15: 3527}


In [ ]:
'''from sklearn.model_selection import train_test_split

# train, valid 분할
data = train_df[['reviews_hanspell','label']]
dataset_train, dataset_valid = train_test_split(data, test_size=0.2, random_state=42)

# 내용 확인
print(dataset_train.iloc[0])
print(dataset_train.iloc[-1])'''

"from sklearn.model_selection import train_test_split\n\n# train, valid 분할\ndata = train_df[['reviews_hanspell','label']]\ndataset_train, dataset_valid = train_test_split(data, test_size=0.2, random_state=42)\n\n# 내용 확인\nprint(dataset_train.iloc[0])\nprint(dataset_train.iloc[-1])"

In [ ]:
'''
def removing_non_korean(df):
    for idx, row in tqdm(df.iterrows(), desc='removing_non_korean', total=len(df)):
        new_doc = re.sub('[^가-힣]', '', row['data']).strip()
        df.loc[idx, 'data'] = new_doc
    return df

train = removing_non_korean(train)
test = removing_non_korean(test)'''

"\ndef removing_non_korean(df):\n    for idx, row in tqdm(df.iterrows(), desc='removing_non_korean', total=len(df)):\n        new_doc = re.sub('[^가-힣]', '', row['data']).strip()\n        df.loc[idx, 'data'] = new_doc\n    return df\n\ntrain = removing_non_korean(train)\ntest = removing_non_korean(test)"

In [ ]:
#!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [ ]:
'''tags = ['JK', 'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ', 'JX', 'JC', 'EP', 'EF', 'EC', 'ETN', 'ETM']

from konlpy.tag import Mecab

m = Mecab()

def remove_josa_mecab(df, tags):
    for idx, row in tqdm(df.iterrows(), desc='removing josa', total=len(df)):
        josa_removed = [x[0] for x in m.pos(row['data']) if x[1] not in tags]
        df.loc[idx, 'data'] = ' '.join(josa_removed)
    return df

train = remove_josa_mecab(train, tags)
test = remove_josa_mecab(test, tags)'''

"tags = ['JK', 'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ', 'JX', 'JC', 'EP', 'EF', 'EC', 'ETN', 'ETM']\n\nfrom konlpy.tag import Mecab\n\nm = Mecab()\n\ndef remove_josa_mecab(df, tags):\n    for idx, row in tqdm(df.iterrows(), desc='removing josa', total=len(df)):\n        josa_removed = [x[0] for x in m.pos(row['data']) if x[1] not in tags]\n        df.loc[idx, 'data'] = ' '.join(josa_removed)\n    return df\n\ntrain = remove_josa_mecab(train, tags)\ntest = remove_josa_mecab(test, tags)"

In [ ]:
model_name = 'monologg/koelectra-small-v3-discriminator'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# Setting parameters
max_len = 128          # 문장의 길이 (평균 13정도)
batch_size = 128
num_epochs = 10
log_interval = 200    # metrics 생성 시점

# 모델에 저장되어 있는 부분들
# learning_rate = 5e-5
# warmup_ratio = 0.1
# max_grad_norm = 1

In [ ]:
# 학습 데이터
# ==> 토크나이징 + 패딩 : dim=64
encoded_train = tokenizer(
    dataset_train['data'].tolist(),
    return_tensors='pt',
    max_length=max_len,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

# 확인
print( encoded_train[0].tokens )
print( encoded_train[0].ids )
print( encoded_train[0].attention_mask )
print()
print('디코딩 :',tokenizer.decode(encoded_train[0].ids))

['[CLS]', '원', '##영', '##이', '광고', '##모', '##델', '##로', '많이', '나오', '##는데', '솔직히', '로봇', '##같', '##음', 'ㅇㅇ', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 

In [ ]:
# 검증 데이터
# ==> 토크나이징 + 패딩 : dim=64
encoded_valid = tokenizer(
    dataset_valid['data'].tolist(),
    return_tensors='pt',
    max_length=max_len,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

print('디코딩 :',tokenizer.decode(encoded_valid[0].ids))
print('디코딩 :',tokenizer.decode(encoded_valid[-1].ids))

디코딩 : [CLS] 인티제가 제일 불쌍한 유형이지 ㅋㅋ 사회생활을 아예 못하니 똑똑하던 일 잘하던 사회생활이 인생에 매번 차지하는데 이게 안되니 진짜 최악이겠네 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
디코딩 : [CLS] 전여친때매 힘든데 너무 깊숙히 박혀있다 나는 되게 신경질적이고 예민하고 화가 많고 직설적이었는데전여자친구 사귀면서 정말 성격 다 죽였거든 이제 참을인도 열까지 셀줄알고 짜증나도 화나도 티도 안내고 말도 이쁘게 하는법 배웠어 여자친구 상처받는게 싫어서그런데 정작 나는 전여자친구한테 너무 큰 상처받고 헤어져서 잊어버리고싶은데나자신을 너무 바꿔놔서 참,, 힘드네 [SEP] [PAD] [PAD] [PAD] [PAD]


In [ ]:
# y_train = np.array( dataset_train['label'].values, dtype='int' )
# y_valid = np.array( dataset_valid['label'].values, dtype='int' )

# 학습 시작 전에 label 데이터가 int형에 1차원인지 확인
print(train['label'].values.shape, dataset_train['label'].values.dtype)
print(test['label'].values.shape, dataset_valid['label'].values.dtype, end='\n\n')
train['label'].values[0]

# 아닐 경우 ==> ValueError: 
# Target size (torch.Size([64])) must be the same as input size (torch.Size([64, 5]))

(45691,) int64
(15231,) int64



11

In [ ]:
from torch.utils.data import Dataset, DataLoader

# 데이터셋 클래스
class ReviewDataset(Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = { key: torch.tensor(val[idx]) for key, val in self.encodings.items() }
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)


# 데이터셋 생성
train_dataset = ReviewDataset(encoded_train, dataset_train['label'].values)
valid_dataset = ReviewDataset(encoded_valid, dataset_valid['label'].values)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=16)
model.to(device)

Some weights of the model checkpoint at monologg/koelectra-small-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_

In [ ]:
# 성능 시각화 wandb
# https://docs.wandb.ai/guides/integrations/huggingface

# for report_to
# import wandb
# wandb.init(project="review_ratings-kcelectra")

# 학습 파라미터
training_args = TrainingArguments(
    output_dir='./temp/electra',
    overwrite_output_dir='True',

    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
 
    logging_dir='./temp/logs',
    logging_steps=log_interval,
    evaluation_strategy="steps",
    eval_steps=log_interval,

    # https://discuss.huggingface.co/t/save-only-best-model-in-trainer/8442/8
    save_total_limit=2,
    save_strategy='no',
    load_best_model_at_end=False,

    # 성능 그래프 시각화 유틸리티 (사용안함)
    # report_to="wandb",
    # run_name="transformer-kcelectra_1"
)

#정확도 측정을 위한 함수 정의
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # average: 'micro', 'macro', 'weighted' or 'samples' 
    # 참고 https://aimb.tistory.com/152
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'acc': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

# wandb.finish()   # 리포트 종료: report_to

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 45691
  Num Epochs = 10
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 3570
  Number of trainable parameters = 14126096
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


Step,Training Loss,Validation Loss,Acc,F1,Precision,Recall
200,2.655000,2.467889,0.223623,0.149140,0.128850,0.223623
400,2.310500,2.144858,0.320137,0.245738,0.269200,0.320137
600,2.068000,1.946161,0.425973,0.391527,0.448586,0.425973
800,1.879400,1.768961,0.488674,0.482223,0.532508,0.488674
1000,1.748300,1.689239,0.504826,0.503387,0.529228,0.504826
1200,1.658700,1.641121,0.512704,0.508991,0.539269,0.512704
1400,1.619100,1.603778,0.524982,0.529193,0.550373,0.524982
1600,1.562600,1.594404,0.526295,0.525182,0.542857,0.526295
1800,1.539800,1.587507,0.522356,0.523718,0.545401,0.522356
2000,1.497500,1.560439,0.535290,0.541067,0.562118,0.535290


***** Running Evaluation *****
  Num examples = 15231
  Batch size = 128
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
***** Running Evaluation *****
  Num examples = 15231
  Batch size = 128
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` paramete

TrainOutput(global_step=3570, training_loss=1.6618402646703212, metrics={'train_runtime': 1930.2225, 'train_samples_per_second': 236.714, 'train_steps_per_second': 1.85, 'total_flos': 3361794586091520.0, 'train_loss': 1.6618402646703212, 'epoch': 10.0})

In [ ]:
trainer.evaluate(valid_dataset)

"""
{'epoch': 5.0,
 'eval_acc': 0.6546,
 'eval_f1': 0.6546,
 'eval_loss': 1.5792670249938965,
 'eval_precision': 0.6546,
 'eval_recall': 0.6546,
 'eval_runtime': 8.7521,
 'eval_samples_per_second': 571.294,
 'eval_steps_per_second': 9.026}

{'epoch': 10.0,
 'eval_acc': 0.6360544217687075,
 'eval_f1': 0.6368239894438272,
 'eval_loss': 1.6639291048049927,
 'eval_precision': 0.6398635203587097,
 'eval_recall': 0.6360544217687075,
 'eval_runtime': 9.6302,
 'eval_samples_per_second': 518.994,
 'eval_steps_per_second': 8.203}
 """

***** Running Evaluation *****
  Num examples = 15231
  Batch size = 128
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
torch.save(model.state_dict(), os.path.join(path,"koembtiweight2.pt"))
#model.load_state_dict(torch.load('model.pth'))

torch.save(model, os.path.join(path,"koembtifullmodel2.pt"))
#torch.load('model.pth')